In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

In [ ]:
# LDDMM landmark two-jet dynamics

In [ ]:
#import src.params as params
#params.manifold = 'landmarks'
from src.manifolds.landmarks import *
M = landmarks(2)
print(M)

from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
# example configuration
M.k_sigma.set_value(np.diag((.5,.5)))

x = np.array([[0.5,1],[1,1]])
q = x.flatten()
v = np.array([[0.,1],[0.,1]])
p = M.flatf(q,v.flatten())

print("q = ", q)
print("p = ", p)

In [ ]:
# regular 0th order Hamiltonian dynamics
from src.dynamics import Hamiltonian
Hamiltonian.initialize(M)
# test Hamiltionian and gradients
print(p)
print(M.Hf(q,p))

# geodesic
qs = M.Exp_Hamiltoniantf(q,p).T
M.plot()
M.plotx(qs,v)
plt.show()
(ts,qps) = M.Hamiltonian_dynamicsf(q,p)
ps = qps[:,1,:]
print("Energy: ",np.array([M.Hf(q,p) for (q,p) in zip(qs,ps)]))

In [ ]:
from src.dynamics import jets
jets.initialize(M)

# test
qqsv = M.Ham_advectf(np.vstack((q.reshape((-1,M.m.eval())),np.array([[0.6,1],[.9,1]]))),
                   qs,ps)[1].reshape((n_steps.eval(),-1))
M.plotx(qqsv)
# grid
(grid,Nx,Ny) = M.getGrid(-.25,1.75,0.,2.,xpts=50,ypts=50)
mgrid = M.Ham_advectf(grid,qs,ps)[1][-1]
M.plotGrid(mgrid,Nx,Ny,coloring=True)
plt.axis('equal')
plt.show()
pass

In [ ]:
# WHAT SHOULD BE DONE WITH THIS???

# 2nd order jet Hamiltonian dynamics
def Hjet(q,p,mu1,mu2):
    Kq = K(q,theano.gradient.disconnected_grad(q)).reshape((N,m,N,m)).dimshuffle((0,2,1,3)) # shape=((N,N,m,m)) (i,j,alpha,beta)
    #Kq = K(q,q).reshape((N,m,N,m)).dimshuffle((0,2,1,3)) # shape=((N,N,m,m)) (i,j,alpha,beta)
    #r_sq = T.square((q.dimshuffle(0,'x',1)-q.dimshuffle('x',0,1))).sum(2)
    #Ksigma = r_sq#T.exp( - r_sq / (2*SIGMA**2) )
    #DKq = T.jacobian((
    #    (Ksigma[:,:,np.newaxis,np.newaxis]*T.eye(m)[np.newaxis,np.newaxis,:,:]).dimshuffle((0,2,1,3)).reshape((-1,d))
    #).flatten(),q).sum(1).reshape((N,N,m,m,m)) # (i,j,alpha,beta,gamma)
    DKq = T.jacobian(Kq.flatten(),q).sum(1).reshape((N,N,m,m,m)) # (i,j,alpha,beta,gamma)    
    #D2Kq = T.jacobian(DKq.flatten(),q).sum(1).reshape((N,N,m,m,m,m)) # (i,j,alpha,beta,gamma,delta)
    #D3Kq = T.jacobian(D2Kq.flatten(),q).sum(1).reshape((N,N,m,m,m,m,m)) # (i,j,alpha,beta,gamma,delta,epsilon)
    #D4Kq = T.jacobian(D3Kq.flatten(),q).sum(1).reshape((N,N,m,m,m,m,m,m)) # (i,j,alpha,beta,gamma,delta,epsilon,phi)
    
    return (  .5*T.tensordot(p,T.tensordot(p,Kq,((0,1),(2,3))),((0,1),(0,1)))
             - T.tensordot(p,T.tensordot(mu1,DKq,((0,1,2),(2,3,4))),((0,1),(0,1)))  +0*mu2[0,0,0,0] )
             #+ T.tensordot(p,T.tensordot(mu2,D2Kq,((0,1,2,3),(2,3,4,5))),((0,1),(0,1))) )
#           - .5*T.tensordot(mu1,T.tensordot(mu1,D2Kq,((0,1,2),(2,3,4))),((0,1,2),(0,1,2)))
#              + T.tensordot(mu1,T.tensordot(mu2,D3Kq,((0,1,3,2),(2,3,4,5))),((0,1,2),(0,1,2))) )
#           + .5*T.tensordot(mu2,T.tensordot(mu2,D4Kq,((0,1,3,2),(2,3,6,7))),((0,1,2,3),(0,1,2,3))) )
q = T.matrix()
p = T.matrix()
mu1 = T.tensor3()
mu2 = T.tensor4()
Hjetf = theano.function([p,q,mu1,mu2],Hjet(p,q,mu1,mu2),mode='FAST_COMPILE',on_unused_input='ignore')

def coad(xi,mu):
    (xi1,xi2) = xi
    (mu1,mu2) = mu
    return (
        (mu1[:,:,np.newaxis,:]*xi1[:,np.newaxis,:,:]).sum(3)
           -(xi1[:,:,:,np.newaxis]*mu1[:,:,np.newaxis,:]).sum(1)
          +(mu2[:,:,np.newaxis,:,:]*xi2[:,np.newaxis,:,:,:]).sum((3,4))
          -(xi2[:,:,:,np.newaxis,:]*mu2[:,:,np.newaxis,:,:]).sum((1,4))
          -(xi2[:,:,:,:,np.newaxis]*mu2[:,:,:,np.newaxis,:]).sum((1,2))
        ,
        (mu2[:,:,np.newaxis,:,:]*xi1[:,np.newaxis,:,:,np.newaxis]).sum(3)
            +(mu2[:,:,np.newaxis,:,:]*xi1[:,np.newaxis,:,:,np.newaxis]).sum(3)
            -(xi1[:,:,:,np.newaxis,np.newaxis]*mu2[:,:,np.newaxis,:,:]).sum(1)
    )
xi1 = T.tensor3()
xi2 = T.tensor4()
coadf = theano.function([xi1,xi2,mu1,mu2],coad((xi1,xi2),(mu1,mu2)),mode='FAST_COMPILE',on_unused_input='ignore')

def state_to_weinstein_darboux(x):
    q = x[0:N*m].reshape((N,m))
    p = x[N*m:N*m+N*m].reshape((N,m))
    mu1 = x[N*m+N*m:N*m+N*m+N*m*m].reshape((N,m,m))
    mu2 = x[N*m+N*m+N*m*m:N*m+N*m+N*m*m+N*m*m*m].reshape((N,m,m,m))
    return (q,p,mu1,mu2)

def weinstein_darboux_to_state(q,p,mu1,mu2):
    return T.stack((q.flatten(),p.flatten(),mu1.flatten(),mu2.flatten()))
    
def ode_Ham_jet(t,x):
    dq  = lambda q,p,mu1,mu2: T.grad(Hjet(q,p,mu1,mu2),p)
    dp  = lambda q,p,mu1,mu2: -T.grad(Hjet(q,p,mu1,mu2),q)
    xi  = lambda q,p,mu1,mu2: (T.grad(Hjet(q,p,mu1,mu2),mu1),T.grad(Hjet(q,p,mu1,mu2),mu2))
    negdmu = lambda q,p,mu1,mu2: coad(xi(q,p,mu1,mu2),(mu1,mu2))

    (q,p,mu1,mu2) = state_to_weinstein_darboux(x)
    dqt = dq(p,q,mu1,mu2)
    dpt = dp(p,q,mu1,mu2)
    (negdmu1t,negdmu2t) = negdmu(q,p,mu1,mu2)
    return weinstein_darboux_to_state(dqt,dpt,-negdmu1t,-negdmu2t)
Ham = lambda q,p,mu1,mu2: integrate(ode_Ham_jet,weinstein_darboux_to_state(q,p,mu1,mu2))
Hamf = theano.function([q,p,mu1,mu2], Ham(q,p,mu1,mu2),mode='FAST_COMPILE',on_unused_input='ignore')